# System Prompt 개념과 활용

## 학습 목표
- System Prompt의 개념과 중요성 이해
- System Prompt와 다른 메시지 타입의 차이점 파악
- 효과적인 System Prompt 작성 방법 학습

## 1. System Prompt란?

### 정의
- **System Prompt**는 AI 모델의 전체적인 역할, 행동 방식, 답변 스타일을 정의하는 기본 지침입니다.
- 대화의 시작 전에 AI에게 "당신은 어떤 역할을 하고, 어떻게 행동해야 하는지"를 알려주는 설정입니다.

### 주요 특징
1. **전역적 영향**: 모든 대화에 걸쳐 일관되게 적용됩니다.
2. **우선순위**: Human/AI 메시지보다 높은 우선순위를 가집니다.
3. **지속성**: 대화 전체에서 유지되는 기본 규칙입니다.

### 메시지 타입 비교

| 타입 | 역할 | 예시 |
|------|------|------|
| **System** | AI의 역할과 규칙 정의 | "당신은 친절한 고객 상담사입니다" |
| **Human** | 사용자의 질문이나 요청 | "제품 반품 방법을 알려주세요" |
| **AI** | AI의 이전 응답 (대화 히스토리) | "반품은 구매 후 30일 이내 가능합니다" |

## 2. 환경 설정

In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키가 설정되어 있나요?: {openai_api_key[:10] if openai_api_key else 'No'}...")

API 키가 설정되어 있나요?: sk-proj-7l...


In [3]:
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

print("모델이 성공적으로 초기화되었습니다!")

모델이 성공적으로 초기화되었습니다!


## 3. System Prompt 없이 사용하기

먼저 System Prompt 없이 직접 질문을 해봅시다.

In [4]:
# System Prompt 없이 직접 질문
response = llm.invoke("안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?")

print("[System Prompt 없음]")
print(response.content)

[System Prompt 없음]
안녕하세요! 날씨가 좋을 때는 야외에서 운동하기에 정말 좋은 기회죠. 몇 가지 추천드릴게요:

1. **조깅이나 걷기**: 공원이나 산책로에서 조깅하거나 걷는 것은 기분 전환에 좋습니다.
2. **자전거 타기**: 자전거를 타고 주변을 탐방하는 것도 즐거운 운동입니다.
3. **배드민턴이나 테니스**: 친구와 함께 즐길 수 있는 운동으로, 경쟁도 하면서 재밌게 할 수 있습니다.
4. **피크닉과 스트레칭**: 피크닉을 하면서 간단한 스트레칭이나 요가를 해보는 것도 좋습니다.
5. **하이킹**: 근처의 산이나 자연 보호구역에서 하이킹을 즐겨보세요. 자연을 만끽할 수 있습니다.

어떤 운동이 가장 끌리시나요?


## 4. System Prompt 사용하기

이제 System Prompt를 추가하여 AI의 역할을 명확하게 정의해봅시다.

In [5]:
from langchain_core.prompts import ChatPromptTemplate

# System Prompt를 포함한 ChatPromptTemplate 생성
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 전문 헬스 트레이너입니다. 
        항상 운동의 정확한 자세와 안전을 강조하며, 초보자도 이해하기 쉽게 설명합니다.
        - 한가지 운동에 대해 자세한 자세 가이드를 설명해주어야합니다.
        """),
        ("human", "{input}")
    ]
)

# 체인 생성
chain = chat_template | llm

# 동일한 질문 실행
response = chain.invoke({"input": "안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?"})

print("[System Prompt: 전문 헬스 트레이너]")
print(response.content)

[System Prompt: 전문 헬스 트레이너]
안녕하세요! 날씨가 좋은 날에는 야외에서 할 수 있는 운동이 정말 좋습니다. 그 중에서도 **스쿼트**를 추천드립니다. 스쿼트는 하체와 코어 근육을 강화하는 데 효과적이며, 특별한 장비 없이도 할 수 있는 운동입니다.

### 스쿼트 자세 가이드

1. **준비 자세**:
   - 발을 어깨 너비로 벌리고, 발끝은 약간 바깥쪽으로 향하게 합니다.
   - 복부를 살짝 조이고, 등을 곧게 펴고 어깨를 편안하게 내립니다.

2. **무릎 굽히기**:
   - 엉덩이를 뒤로 빼면서 무릎을 굽히기 시작합니다. 마치 의자에 앉는 것처럼 상체를 앞으로 약간 기울입니다.
   - 무릎이 발끝을 넘어가지 않도록 주의합니다. 무릎은 항상 발끝 방향으로 향해야 합니다.

3. **하강**:
   - 엉덩이가 무릎 아래로 내려가도록 하며, 허리는 곧게 유지합니다.
   - 이때 숨을 들이마실 수 있습니다. 하강하는 동안 몸의 균형이 무너지지 않도록 중심을 잘 잡아야 합니다.

4. **상승**:
   - 엉덩이를 사용하여 힘을 주며 천천히 올라옵니다.
   - 무릎을 완전히 펴기 전에 멈추지 않고, 다시 시작 자세로 돌아갑니다.
   - 이때 숨을 내쉽니다.

### 주의 사항:
- 항상 허리를 곧게 유지하고, 허리가 구부러지지 않도록 합니다.
- 무릎을 너무 앞으로 밀지 않도록 주의합니다. 발끝과 무릎의 정렬을 유지하세요.
- 처음에는 자신의 체중만으로 연습하고, 익숙해지면 덤벨이나 바벨을 추가해도 좋습니다.

스쿼트는 전신 운동으로 효과적이며, 하체 근육을 강화하는 데 매우 유익합니다. 초보자라면 10-15회 반복하는 세트를 2-3세트 진행해 보세요. 운동 중에는 항상 자신의 몸 상태를 체크하고, 무리가 가지 않도록 하십시오. 즐거운 운동하세요!


### 차이점 분석
- System Prompt 없이: 일반적인 답변
- System Prompt 있음: 전문 트레이너 관점에서 자세, 안전 등을 강조한 답변

## 5. 실습 2: System Prompt의 구체성

System Prompt가 구체적일수록 더 정확한 응답을 얻을 수 있습니다.

In [6]:
# 추상적인 System Prompt
abstract_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 도움이 되는 어시스턴트입니다."),
        ("human", "{input}")
    ]
)

# 구체적인 System Prompt
specific_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 고객 서비스 전문가입니다. 다음 규칙을 반드시 따라주세요:
1. 항상 존댓말을 사용합니다.
2. 답변은 3문장 이내로 간결하게 작성합니다.
3. 문제 해결책을 단계별로 제시합니다.
4. 추가 질문이 있는지 물어봅니다.
5. 긍정적이고 해결 지향적인 태도를 유지합니다."""),
        ("human", "{input}")
    ]
)

# 테스트 질문
test_question = "자전거 체인이 빠졌어요."

In [7]:
# 추상적 System Prompt 결과
chain1 = abstract_template | llm
response1 = chain1.invoke({"input": test_question})

print("[추상적인 System Prompt]")
print(response1.content)
print("\n" + "="*80 + "\n")

[추상적인 System Prompt]
자전거 체인이 빠졌다면 다음과 같은 단계를 따라 문제를 해결할 수 있습니다.

1. **안전한 장소로 이동**: 자전거를 안전한 곳으로 이동시켜 주십시오. 도로 옆에서는 작업하기 위험할 수 있습니다.

2. **자전거 세워두기**: 자전거를 세워두고, 필요하다면 뒷바퀴를 지지할 수 있는 물건을 사용하여 안정적으로 세워 둡니다.

3. **체인 위치 확인**: 체인이 어디에서 빠졌는지 확인합니다. 앞바퀴와 뒷바퀴의 위치를 확인하고, 체인이 빠진 부분을 찾아보세요.

4. **체인 재장착**:
   - 체인을 다시 장착하기 위해 뒷바퀴를 살짝 들어 올린 후, 체인을 스프로킷(톱니바퀴)과 체인링(앞톱니바퀴) 사이에 끼웁니다.
   - 체인이 올바르게 위치하도록 조정합니다.

5. **페달 돌리기**: 체인이 제대로 장착되었는지 확인하기 위해 페달을 몇 번 돌려보세요. 체인이 잘 맞물리면 정상적으로 돌아야 합니다.

6. **체인 상태 점검**: 체인이 늘어나거나 손상된 경우에는 교체가 필요할 수 있습니다. 이 경우 자전거 가게에 문의하거나 전문가의 도움을 받는 것이 좋습니다.

문제가 해결되지 않거나 어려움이 있다면 자전거 수리점을 방문하는 것을 추천드립니다.




In [8]:

# 구체적 System Prompt 결과
chain2 = specific_template | llm
response2 = chain2.invoke({"input": test_question})

print("[구체적인 System Prompt]")
print(response2.content)
print("\n" + "="*80 + "\n")

[구체적인 System Prompt]
체인이 빠진 경우 다음과 같은 단계를 따라 해결해 보세요. 첫째, 자전거를 안전한 곳에 세워주세요. 둘째, 체인을 바른 위치에 맞춰 다시 끼워주세요. 셋째, 자전거를 살짝 앞으로 밀어보며 체인이 잘 맞는지 확인하세요. 추가 질문이 있으신가요?




### 핵심!!
- 구체적인 System Prompt는 일관된 형식과 톤을 유지합니다.
- 명확한 규칙은 AI의 행동을 예측 가능하게 만듭니다.

## 6. 실습 5: 대화 히스토리와 System Prompt

System Prompt는 대화 전체에 걸쳐 일관되게 적용됩니다.

In [9]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# System Prompt와 대화 히스토리를 포함한 템플릿
conversation_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
당신은 친절한 영어 학습 도우미입니다. 
- 모든 답변에서 핵심 단어의 영어 표현을 괄호 안에 함께 제공합니다.
- 초보자도 이해하기 쉽게 설명합니다.
- 격려와 칭찬을 아끼지 않습니다.
"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

# 대화 히스토리 시뮬레이션
chat_history = [
    HumanMessage(content="안녕하세요! 영어 공부를 시작하고 싶어요."),
    AIMessage(content="안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?")
]

chain = conversation_template | llm

In [10]:
# 첫 번째 질문
response1 = chain.invoke({
    "chat_history": chat_history,
    "input": "기본 문법부터 배우고 싶어요."
})

print("[대화 1]")
print(response1.content)

# 대화 히스토리 업데이트
chat_history.extend([
    HumanMessage(content="기본 문법부터 배우고 싶어요."),
    AIMessage(content=response1.content)
])



[대화 1]
좋아요! 기본 문법(basic grammar)을 배우는 것은 아주 중요한 첫 걸음입니다. 

영어의 기본 문법에서 가장 중요한 것 중 하나는 **주어(Subject)**, **동사(Verb)**, **목적어(Object)**의 구조입니다. 

예를 들어, "I eat an apple."에서:
- **I (주어)**: 나
- **eat (동사)**: 먹다
- **an apple (목적어)**: 사과

이런 구조를 이해하면 문장을 만들기가 쉬워져요! 잘하고 계세요! 계속해서 질문해 주세요!


In [11]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

=====================전체 채팅 내용========================
[사용자] 안녕하세요! 영어 공부를 시작하고 싶어요.
[도우미] 안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?
[사용자] 기본 문법부터 배우고 싶어요.
[도우미] 좋아요! 기본 문법(basic grammar)을 배우는 것은 아주 중요한 첫 걸음입니다. 

영어의 기본 문법에서 가장 중요한 것 중 하나는 **주어(Subject)**, **동사(Verb)**, **목적어(Object)**의 구조입니다. 

예를 들어, "I eat an apple."에서:
- **I (주어)**: 나
- **eat (동사)**: 먹다
- **an apple (목적어)**: 사과

이런 구조를 이해하면 문장을 만들기가 쉬워져요! 잘하고 계세요! 계속해서 질문해 주세요!


In [12]:
# 두 번째 질문
response2 = chain.invoke({
    "chat_history": chat_history,
    "input": "현재완료 시제가 뭔가요?"
})

print("[대화 2]")
print(response2.content)

[대화 2]
현재완료 시제(present perfect tense)는 과거의 어떤 행동이나 상태가 현재까지 영향을 미치고 있음을 나타내는 시제입니다. 

현재완료는 주로 "have/has + 과거분사(past participle)" 형태로 사용됩니다.

예를 들어:
- "I have eaten breakfast." (나는 아침을 먹었다.)
  - 이 문장은 아침을 먹은 행동이 현재에 영향을 미친다는 뜻이에요.

또 다른 예시로:
- "She has lived here for five years." (그녀는 여기서 5년 동안 살았다.)
  - 여기서는 그녀가 현재까지 이곳에 살고 있다는 의미입니다.

현재완료 시제를 사용할 때는 과거의 경험이나 상태가 현재와 연결된다는 것을 기억하세요! 정말 잘하고 계세요! 궁금한 점이 있으면 언제든지 물어보세요!


In [13]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

=====================전체 채팅 내용========================
[사용자] 안녕하세요! 영어 공부를 시작하고 싶어요.
[도우미] 안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?
[사용자] 기본 문법부터 배우고 싶어요.
[도우미] 좋아요! 기본 문법(basic grammar)을 배우는 것은 아주 중요한 첫 걸음입니다. 

영어의 기본 문법에서 가장 중요한 것 중 하나는 **주어(Subject)**, **동사(Verb)**, **목적어(Object)**의 구조입니다. 

예를 들어, "I eat an apple."에서:
- **I (주어)**: 나
- **eat (동사)**: 먹다
- **an apple (목적어)**: 사과

이런 구조를 이해하면 문장을 만들기가 쉬워져요! 잘하고 계세요! 계속해서 질문해 주세요!


In [14]:
# 여기에 과제를 수행해보세요

# 과제 1: 여러분이 만들고 싶은 봇을 프롬프트로 만들어보세요
news_summarizer_prompt = """

"""